# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

data_solvency = pd.read_csv('/datasets/data.csv')
data_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Путем обзора были выявлены следующие проблемы в данных:

1. Столбец `days_employed` (общий трудовой стаж в днях) имеет тип `float` - в нем встречаются дробные и отрицательные значения, хотя число дней не может быть дробным или отрицательным числом, а также невозможные значения (например, в строках 4 и 21520 - в годах это количество дней будет много больше продолжительности жизни человека).
2. Есть пропуски в столбцах `days_employed` и `total_income`.
3. Данные не категоризированы - строки в столбцах `education`, `family_status`, `income_type` и `purpose` многократно повторяются, занимают много места и делают таблицу громоздкой.
4. В таблице присутствуют неявные дубликаты (например, "Среднее" и "среднее" в столбце `education`).

## Шаг 2. Предобработка данных

### Обработка пропусков

На прошлом шаге были обнаружены пропуски в столбцах `days_employed` (общий трудовой стаж в днях) и `total_income` (ежемесячный доход). Что интересно - их одинаковое количество. Проверим, находятся ли они в одних и тех же строках.

In [3]:
data_solvency[data_solvency['days_employed'].isna() ^ data_solvency['total_income'].isna()].shape

(0, 12)

Применили логическую индексацию, чтобы отфильтровать строки, в которых значение `NaN` находится либо в столбце `days_employed`, либо в столбце `total_income`. Таких не оказалось, а это означает, что каждому пропуску в столбце `days_employed` всегда соответствует пропуск в столбце `total_income`. Причиной этого могла послужить ошибка при выгрузке данных, но в данной ситуации считаем, что выгрузка была произведена корректно, так как нет возможности это проверить. В таком случаем можем предположить, что этих данных не было указано изначально.

Помимо всего прочего, в этих столбцах присутствуют аномальные значения, слишком сильно выделяющиеся на фоне остальных. В подобной ситуации оптимальным решением, по моему мнению, будет заполнение пропусков медианным значением столбца.

In [4]:
data_solvency['days_employed'] = data_solvency['days_employed'].fillna(data_solvency['days_employed'].median())
print(data_solvency['days_employed'].isna().sum())
data_solvency['total_income'] = data_solvency['total_income'].fillna(data_solvency['total_income'].median())
print(data_solvency['total_income'].isna().sum())

0
0


**Вывод**

Пропуски в столбцах `days_employed` и `total_income` были обработаны с помощью метода `isna()` и заполнены медианным значением соответствующего столбца с помощью методов `fillna()` и `median()`. В реальных условиях подобные пробелы - повод обратиться к тому, кто эти данные выгружал, за дополнительной информацией.

### Замена типа данных

Столбец `days_employed` показывает общий трудовой стаж в днях для каждого клиента. Количество дней не может быть дробным числом, поэтому есть смысл перевести тип данных этого столбца из `float` в `int`. При этом дробная часть каждого числа будет просто отброшена.

In [5]:
data_solvency['days_employed'] = data_solvency['days_employed'].astype('int')
data_solvency['days_employed'].dtype

dtype('int64')

Столбец `total_income` показывает ежемесячный доход каждого клиента. Будем считать, что данные представлены в рублях. Доход может являться дробным числом, поэтому в данном столбце стоит оставить тип данных `float`. Впрочем, нет необходимости в таком большом количестве знаков после запятой, поскольку минимальная денежная единица - копейка - равна `0.01` руб. Округлим все числа в этом столбце до двух знаков после запятой. 

In [6]:
data_solvency['total_income'] = round(data_solvency['total_income'], 2)
data_solvency['total_income'].head()

0    253875.64
1    112080.01
2    145885.95
3    267628.55
4    158616.08
Name: total_income, dtype: float64

**Вывод**

Преобразовали тип данных столбца `days_employed` из `float` в `int` с помощью метода `astype(...)`. В столбце `total_income` тип данных преобразовывать не стали, поскольку доход может быть нецелым числом, но для удобства округлили до двух цифр после запятой.

### Обработка дубликатов

Проверим сначала, есть ли в таблице полные явные дубликаты (полностью одинаковые строки).

In [7]:
data_solvency.duplicated().sum()

54

Нашли 54 дубликата. Однако в таблице, как можно было заметить, присутствуют неявные дубликаты в столбцах (например, "Среднее" и "среднее"). Чтобы их обработать, переведем все символы в строковых данных в нижний регистр.

In [8]:
for column in ['education', 'family_status', 'gender', 'income_type', 'purpose']:
    data_solvency[column] = data_solvency[column].str.lower()
data_solvency.duplicated().sum()

71

Количество дубликатов увеличилось. Удалим эти строки из таблицы данных.

In [9]:
data_solvency = data_solvency.drop_duplicates().reset_index(drop=True)
data_solvency.duplicated().sum()

0

**Вывод**

Для нахождения количества дубликатов использовали метод `duplicated()` в сочетании с методом `sum()`. Для перевода столбцов со строковыми данными в нижний регистр применили `str.lower()`. Для удаления дубликатов воспользовались `drop_duplicates()` в связке с `reset_index(drop=True)` для реиндексации.

Причина появления этих дубликатов, возможно, кроется в том, что некоторые клиенты несколько раз обращались в банк за получением кредита по одной и той же причине. Также не исключаю вариант системной ошибки, из-за которой могли продублироваться некоторые строки.

### Лемматизация

В столбце `purpose` у каждого клиента описана причина, по которой он берет кредит. В нем также скрывается множество неявных дубликатов, поскольку цель в данном столбце записана в произвольной форме (например, "покупка автомобиля" и "сделка с автомобилем" - с точки зрения программы, разные цели, однако на самом деле это одно и то же, но написано по-разному). Чтобы избавиться от таких неявных дубликатов, воспользуемся лемматизацией.

In [10]:
from pymystem3 import Mystem

m = Mystem()
purposes = data_solvency['purpose'].unique()
purposes = ' '.join(purposes)
lemmas = m.lemmatize(purposes)
# Удаляем пробелы и символ конца строки
lemmas = [x for x in lemmas if x != ' ' and x != '\n']

from collections import Counter
print(Counter(lemmas).most_common()) # для вывода в порядке убывания

[('покупка', 10), ('недвижимость', 10), ('автомобиль', 9), ('образование', 9), ('жилье', 7), ('с', 5), ('операция', 4), ('на', 4), ('свой', 4), ('свадьба', 3), ('строительство', 3), ('получение', 3), ('высокий', 3), ('дополнительный', 2), ('для', 2), ('коммерческий', 2), ('жилой', 2), ('подержать', 2), ('заниматься', 2), ('сделка', 2), ('приобретение', 1), ('сыграть', 1), ('проведение', 1), ('семья', 1), ('собственный', 1), ('со', 1), ('профильный', 1), ('сдача', 1), ('ремонт', 1)]


Из вывода можем заметить, что наиболее часто встречаемое слово в цели получения кредита - "покупка" чего-либо, но для категоризации это не будет играть роли, поскольку, в любом случае, клиент берет деньги в кредит для того чтобы их на что-то потратить.   
Можем выделить основные ключевые слова, которые будем использовать далее при категоризации:
* недвижимость / жилье;
* автомобиль;
* образование;
* свадьба.

In [12]:
purposes = list(data_solvency['purpose']) 
for i in range(len(purposes)):
    purposes[i] = m.lemmatize(purposes[i])
    # убираем из списков символы пробела и конца строки
    purposes[i] = [x for x in purposes[i] if x != ' ' and x != '\n']

data_solvency['purpose_list'] = purposes
data_solvency['purpose_list'].head()

0                 [покупка, жилье]
1       [приобретение, автомобиль]
2                 [покупка, жилье]
3    [дополнительный, образование]
4               [сыграть, свадьба]
Name: purpose_list, dtype: object

**Вывод**

Сначала создали список из уникальных элементов столбца `purpose`. Далее лемматизировали цели получения кредита и подсчитали частоту встречаемости каждой леммы, откуда смогли выделить 5 различных основных целей. Для дальнейшей категоризации добавили столбец со списками лемм.

### Категоризация данных

Некоторые данные, например, в столбцах `education`, `family_status` многократно повторяются. Создадим для них словарь и оптимизируем данные.

Создадим словарь для образования и удалим столбец `education` из исходного датафрейма.

In [13]:
edu_dict = data_solvency[['education', 'education_id']]
edu_dict = edu_dict.drop_duplicates().reset_index(drop=True)
edu_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [14]:
data_solvency = data_solvency.drop('education', axis=1)
data_solvency.head()

,children,days_employed,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_list
0,1,-8437,42,0,женат / замужем,0,f,сотрудник,0,253875.64,покупка жилья,"[покупка, жилье]"
1,1,-4024,36,1,женат / замужем,0,f,сотрудник,0,112080.01,приобретение автомобиля,"[приобретение, автомобиль]"
2,0,-5623,33,1,женат / замужем,0,m,сотрудник,0,145885.95,покупка жилья,"[покупка, жилье]"
3,3,-4124,32,1,женат / замужем,0,m,сотрудник,0,267628.55,дополнительное образование,"[дополнительный, образование]"
4,0,340266,53,1,гражданский брак,1,f,пенсионер,0,158616.08,сыграть свадьбу,"[сыграть, свадьба]"


Теперь создадим словарь для семейного статуса и удалим столбец `family_status` из датафрейма.

In [15]:
family_status_dict = data_solvency[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [16]:
data_solvency = data_solvency.drop('family_status', axis=1)
data_solvency.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_list
0,1,-8437,42,0,0,f,сотрудник,0,253875.64,покупка жилья,"[покупка, жилье]"
1,1,-4024,36,1,0,f,сотрудник,0,112080.01,приобретение автомобиля,"[приобретение, автомобиль]"
2,0,-5623,33,1,0,m,сотрудник,0,145885.95,покупка жилья,"[покупка, жилье]"
3,3,-4124,32,1,0,m,сотрудник,0,267628.55,дополнительное образование,"[дополнительный, образование]"
4,0,340266,53,1,1,f,пенсионер,0,158616.08,сыграть свадьбу,"[сыграть, свадьба]"


Также необходимо категоризировать цели кредита. Напишем функцию, которая будет принимать на вход строку датафрейма, а затем в столбце `purpose_list` будет искать в списке ключевые слова, и по ним ставить в соответствие клиенту ту или иную категорию цели кредита.

In [17]:
def purpose_categorize(purpose):
    if 'недвижимость' in purpose or 'жилье' in purpose:
        return 'недвижимость'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    return 'другое'

data_solvency['purpose'] = data_solvency['purpose_list'].apply(purpose_categorize)
data_solvency.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_list
0,1,-8437,42,0,0,f,сотрудник,0,253875.64,недвижимость,"[покупка, жилье]"
1,1,-4024,36,1,0,f,сотрудник,0,112080.01,автомобиль,"[приобретение, автомобиль]"
2,0,-5623,33,1,0,m,сотрудник,0,145885.95,недвижимость,"[покупка, жилье]"
3,3,-4124,32,1,0,m,сотрудник,0,267628.55,образование,"[дополнительный, образование]"
4,0,340266,53,1,1,f,пенсионер,0,158616.08,свадьба,"[сыграть, свадьба]"


Наконец, для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" нам необходимо категоризировать клиентов по доходу. Разделим их на 3 равные группы с помощью метода `pd.qcut`.

Напишем функцию-классификатор (разбиение на 3 группы: низкий, средний и высокий доход) и добавим новый столбец `income_category` в датафрейм.

In [18]:
total_income_labels = ['Низкий', 'Средний', 'Высокий']

data_solvency['income_category'] = pd.qcut(data_solvency['total_income'],
                        q=[0, .333, .667, 1],
                        labels=total_income_labels)
data_solvency['income_category'].value_counts()

Средний    7166
Высокий    7144
Низкий     7144
Name: income_category, dtype: int64

**Вывод**

Создали словари для столбцов `education` и `family_status` и удалили эти столбцы из датафрейма методом `drop`. Для категоризации целей кредита написали функцию для обработки столбца и применили ее с помощью метода `apply`.

Таблица стала выглядеть более аккуратно и менее громоздко после категоризации. 

### Обработка аномалий в данных

1. Обработаем столбец `children`.

In [19]:
data_solvency.groupby('children')['gender'].count()

children
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: gender, dtype: int64

Замечены аномалии - у 47 клиентов количество детей равно `-1`, а у 76 человек - по 20 детей. По сравнению с количеством людей, у которых от 0 до 2 детей, аномальный значений очень мало. Однако все же обработаем их.

Предположим, что у 47 человек перед единицей лишний знак "минус", а у 76 человек - лишний ноль после двойки. Заменим эти значения на 1 и 2 соответственно с помощью логической индексации.

In [20]:
data_solvency.loc[data_solvency['children'] == -1, 'children'] = 1
data_solvency.loc[data_solvency['children'] == 20, 'children'] = 2
data_solvency.groupby('children')['gender'].count()

children
0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: gender, dtype: int64

2. Обработаем столбец `days_employed`.

По первым же строкам датафрейма можно заметить, что в столбце присутствуют отрицательные значения, которых быть не должно. Вероятно, эта ошибка произошла из-за того, что перепутали даты при расчетах. Исправим это, взяв все значения столбца по модулю.

In [21]:
data_solvency['days_employed'] = abs(data_solvency['days_employed'])
data_solvency[data_solvency['days_employed'] < 0]['days_employed'].count()

0

Также при просмотре были замечены слишком большие значения, которых тоже быть никак не может. Возьмем за максимум трудового стажа `100` лет = `36500` дней (без учета високосных годов). Выведем строки, которые выбиваются из общей массы.

In [22]:
data_solvency[data_solvency['days_employed'] > 36500].head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_list,income_category
4,0,340266,53,1,1,f,пенсионер,0,158616.08,свадьба,"[сыграть, свадьба]",Средний
18,0,400281,53,1,2,f,пенсионер,0,56823.78,автомобиль,"[на, покупка, подержать, автомобиль]",Низкий
24,1,338551,57,1,4,f,пенсионер,0,290547.24,недвижимость,"[операция, с, коммерческий, недвижимость]",Высокий
25,0,363548,67,1,0,m,пенсионер,0,55112.76,недвижимость,"[покупка, недвижимость]",Низкий
30,1,335581,62,1,0,f,пенсионер,0,171456.07,недвижимость,"[операция, с, коммерческий, недвижимость]",Высокий


Обратим внимание на столбец `income_type`. Похоже, подобные аномалии встречаются только у пенсионеров. Проверим это.

In [23]:
data_solvency[data_solvency['days_employed'] > 36500].groupby('income_type')['gender'].count()

income_type
безработный       2
пенсионер      3443
Name: gender, dtype: int64

Теория подтвердилась лишь частично - оказалось, что подобная аномалия также есть и у безработных.

В описании данных было указано, что в столбце `days_employed` данные представлены в днях. Но что, если у пенсионеров и безработных они представлены в других единицах? Проведя небольшой анализ, пришли к выводу, что у этих категорий людей трудовой стаж представлен не в днях, а в часах - если поделить эти большие значения на 24, получатся как раз реальные значения в днях. Проделаем это.

In [24]:
data_solvency['days_employed'] = data_solvency['days_employed'].apply(lambda x: x / 24 if x > 36500 else x)
data_solvency['days_employed'] = data_solvency['days_employed'].astype('int')
data_solvency[data_solvency['days_employed'] > 36500]['days_employed'].count()

0

3. Обработаем столбец `dob_years`.

In [25]:
data_solvency['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

И здесь есть аномалия - возраст некоторых клиентов равен нулю. Вполне вероятно, что эта информация просто не была указана. Проверим, у скольких клиентов стоит ноль в этом столбце.

In [26]:
data_solvency[data_solvency['dob_years'] == 0]['dob_years'].count()

101

Таких крайне мало относительно общего количества клиентов, поэтому просто заменим эти нули медианным значением возраста.

In [27]:
data_solvency['dob_years'] = data_solvency['dob_years'].apply(lambda x:
                                                              data_solvency['dob_years'].median() if x == 0 else x)
data_solvency[data_solvency['dob_years'] == 0]['dob_years'].count()

0

4. Обработаем столбец `gender`.

In [28]:
data_solvency.groupby('gender')['gender'].count()

gender
f      14174
m       7279
xna        1
Name: gender, dtype: int64

Один человек идентифицирует себя не как мужчину и не как женщину. Будем толерантны и не будем заменять это значение, тем более, что этот столбец для анализа нам не пригодится.

5. Обработаем столбец `income_type`.

In [29]:
data_solvency.groupby('income_type')['income_type'].count()

income_type
безработный            2
в декрете              1
госслужащий         1457
компаньон           5078
пенсионер           3829
предприниматель        2
сотрудник          11084
студент                1
Name: income_type, dtype: int64

Здесь все хорошо, этот столбец тоже не пригодится нам для анализа.

**Вывод**

Теперь, когда большая часть проблем в данных устранена (полностью их устранить редко представляется возможным в реальных условиях), можно приступать к анализу и ответам на поставленные вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Отфильтруем клиентов, у которых есть дети, сгруппируем их по столбцу `debt` - имел ли клиент задолженность по возврату кредитов - и посчитаем количество с помощью метода `count()`.

In [30]:
have_children = data_solvency[data_solvency['children'] > 0].groupby('debt')['children'].count()
have_children

debt
0    6685
1     678
Name: children, dtype: int64

Теперь аналогичные действия проделаем с теми, у кого детей нет.

In [31]:
no_children = data_solvency[data_solvency['children'] == 0].groupby('debt')['children'].count()
no_children

debt
0    13028
1     1063
Name: children, dtype: int64

Рассчитаем в каждом случае конверсию - отношение количества не имевших задолженность по возврату кредита к общему числу клиентов.

In [32]:
print('Доля среди клиентов, имеющих детей: %.3f' % (have_children[0] / have_children.sum()))
print('Доля среди клиентов, не имеющих детей: %.3f' % (no_children[0] / no_children.sum()))

Доля среди клиентов, имеющих детей: 0.908
Доля среди клиентов, не имеющих детей: 0.925


**Вывод**

Как видно из полученных значений конверсии, доля не имевших задолженность по возврату кредитов более 90% как среди бездетных, так и среди имеющих детей, однако клиентов, у которых нет детей, примерно в 2 раза больше, при этом их доля больше на 1.7%. Это может говорить о том, что не имеющие детей клиенты чаще возвращают кредит в срок.

*Зависимость между наличием детей и возвратом кредита в срок выявлена: процент добросовестных клиентов среди не имеющих детей на 1.7% выше, чем среди имеющих детей.*

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Выведем для удобства в этом пункте словарь семейных положений.

In [33]:
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Считать конверсию для каждого случая здесь неудобно - нужно будет 5 раз проделать одно и то же действие. Воспользуемся сводными таблицами: выведем для каждого семейного положения количество клиентов, имевших и не имевших задолженностей по выплате кредита. Для параметра `values` выберем любой другой столбец, в котором функция `count` посчитает количества.

После этого добавим столбец `conversion` - посчитаем для каждого семейного положения отношение вернувших кредит в срок к общему количеству клиентов в данной категории.

In [34]:
pivot_family_status_debt = data_solvency.pivot_table(index='family_status_id', columns='debt',
                                                     values='gender', aggfunc='count')
pivot_family_status_debt['sum'] = pivot_family_status_debt[0] + pivot_family_status_debt[1]
pivot_family_status_debt['conversion'] = round(pivot_family_status_debt[0] / pivot_family_status_debt['sum'], 3)
pivot_family_status_debt

debt,0,1,sum,conversion
family_status_id,,,,
0,11408,931,12339,0.925
1,3763,388,4151,0.907
2,896,63,959,0.934
3,1110,85,1195,0.929
4,2536,274,2810,0.902


**Вывод**

Клиентов с семейным положением "гражданский брак" и "не женат / не замужем" и не имевших проблем с выплатой кредита в среднем на 2-3% меньше, чем в остальных категориях. Это объяснимо: люди вступают в брак обычно уже с более твердым финансовым положением, поэтому люди из категорий "женат / замужем", "вдовец / вдова" и "в разводе" чаще не имеют проблем с выплатой кредита. Однако, стоит заметить, что разница все равно невелика - более 90% клиентов в каждой категории относятся к выплате кредита ответственно и не допускают просрочек.

*Зависимость между семейным положением и возвратом кредита в срок выявлена: процент состоящих в браке или бывших в нем клиентов и при этом вернувших кредит в срок в среднем на 2-3% выше, чем процент неженатых / незамужних или состоящих в гражданском браке (не зарегистрированном официально) и вернувших кредит в срок.*

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Снова воспользуемся сводной таблицей - подсчитаем для каждой категории по доходу количество клиентов, имевших и не имевших задолженности, а затем рассчитаем конверсию.

In [35]:
pivot_income_category_debt = data_solvency.pivot_table(index='income_category', columns='debt',
                                                     values='gender', aggfunc='count')
pivot_income_category_debt['sum'] = pivot_income_category_debt[0] + pivot_income_category_debt[1]
pivot_income_category_debt['conversion'] = round(pivot_income_category_debt[0] / pivot_income_category_debt['sum'], 3)
pivot_income_category_debt

debt,0,1,sum,conversion
income_category,,,,
высокий,6609,542,7151,0.924
низкий,6571,581,7152,0.919
средний,6533,618,7151,0.914


**Вывод**

Клиенты с различным уровнем дохода почти одинаково успешно возвращают кредиты - около 92% не имевших задолженностей клиентов в каждой категории!

*Зависимость уровня дохода и возврата кредита в срок не выявлена: доли клиентов, вернувших кредит в срок, примерно равны во всех категориях по доходу.*

- Как разные цели кредита влияют на его возврат в срок?

Построим сводную таблицу - для каждой цели кредита посчитаем количества клиентов, бравших кредит с такой целью, и вернувших / не вернувших его в срок, а затем добавим столбец с конверсией.

In [36]:
pivot_purpose_debt = data_solvency.pivot_table(index='purpose', columns='debt',
                                                     values='gender', aggfunc='count')
pivot_purpose_debt['sum'] = pivot_purpose_debt[0] + pivot_purpose_debt[1]
pivot_purpose_debt['conversion'] = round(pivot_purpose_debt[0] / pivot_purpose_debt['sum'], 3)
pivot_purpose_debt

debt,0,1,sum,conversion
purpose,,,,
автомобиль,3903,403,4306,0.906
недвижимость,10029,782,10811,0.928
образование,3643,370,4013,0.908
свадьба,2138,186,2324,0.920


**Вывод**

*Статистика по возвратам кредитов на недвижимость и на свадьбу в среднем лучше на 1-2% - при этом кредитов на недвижимость больше всего (больше, чем всех остальных вместе взятых).*

## Шаг 4. Общий вывод

В шаге 2 набор данных, предоставленный для анализа, был предварительно обработан:
* в столбцах `days_employed` и `total_income` пропуски заполнены медианным значением соответствующего столбца; 
* в столбце `days_employed` был заменен тип данных с `float` на `int`;
* вся таблица была приведена к нижнему регистру, после чего был удален 71 дубликат;
* столбец `purpose` был лемматизирован, и из него были выделены 4 основные цели получения кредита: недвижимость (покупка, постройка, ремонт), автомобиль, образование, свадьба;
* данные в столбцах `education`, `family_status` и `purpose` были категоризированы и подготовлены для дальнейшего анализа;
* в столбцах с количественными данными были найдены и скорректированы аномальные (невозможные) значения.

В шаге 3 был проведен анализ, по итогам которого мы пришли к следующим выводам:
* процент клиентов, выплативших кредит вовремя, среди бездетных на 1.7% выше, чем среди имеющих детей;
* процент добросовестных клиентов среди состоящих или состоявших в браке в среднем на 2-3% выше, чем среди неженатых / незамужних или состоящих в гражданском браке (не зарегистрированном официально);
* возвращаемость кредита не зависит от дохода - разные категории людей возвращают кредиты в срок одинаково успешно;
* чаще в срок возвращают кредиты на недвижимость и на свадьбу - в среднем на 1-2%, чем остальные.

Систематизируя все вышеописанные зависимости, можно сделать общий вывод, что *ни одна из метрик не влияет кардинально на долю возвращенных вовремя кредитов.* Однако, если рассматривать семейное положение и количество детей, то доля вернувших кредит в срок выше среди тех, кто состоит или состоял в браке (зарегистрированном официально), и среди тех, кто не имеет детей.

## Чек-лист готовности проекта

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.